# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

## Store Part I results (from WeatherPy) into DataFrame

* Load the csv exported in Part I to a DataFrame.

In [2]:
weather_csv_file = "../WeatherPy/output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,hasaki,75.0,JP,1.580018e+09,86.0,35.73,140.83,48.00,5.82
1,oktyabrskoye,0.0,RU,1.580018e+09,85.0,43.06,44.74,19.00,6.71
2,atuona,37.0,PF,1.580018e+09,76.0,-9.80,-139.03,81.00,14.47
3,rikitea,24.0,PF,1.580018e+09,78.0,-23.12,-134.97,78.73,11.10
4,bredasdorp,69.0,ZA,1.580018e+09,77.0,-34.53,20.04,63.00,12.75


## Humidity Heatmap

* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
43,napier,0.0,NZ,1.580018e+09,76.0,-39.48,176.92,77.00,4.00
80,arraial do cabo,0.0,BR,1.580018e+09,83.0,-22.97,-42.03,75.20,5.82
163,ilhabela,0.0,BR,1.580018e+09,75.0,-23.78,-45.36,74.30,6.91
227,lalmohan,0.0,BD,1.580018e+09,28.0,22.34,90.74,73.74,7.18
267,bijni,0.0,IN,1.580018e+09,39.0,26.50,90.70,70.11,7.83
297,lambare,0.0,PY,1.580018e+09,88.0,-25.35,-57.61,71.60,3.36
317,san rafael,0.0,AR,1.580018e+09,34.0,-34.62,-68.33,73.62,0.87
462,rosario,0.0,AR,1.580018e+09,78.0,-32.95,-60.64,79.00,3.85


## Hotel Map

* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 43: napier.
Closest hotel is The Nautilus Napier.
------------
Retrieving Results for Index 80: arraial do cabo.
Closest hotel is Pousada Porto Praia.
------------
Retrieving Results for Index 163: ilhabela.
Closest hotel is Ilha Flat Hotel.
------------
Retrieving Results for Index 227: lalmohan.
Closest hotel is Sundorbon courier service.
------------
Retrieving Results for Index 267: bijni.
Closest hotel is Bajwi Hotel Cum Dhaba.
------------
Retrieving Results for Index 297: lambare.
Closest hotel is Depto Tipo Loft Moderno Balcón.
------------
Retrieving Results for Index 317: san rafael.
Closest hotel is Hotel San Rafael.
------------
Retrieving Results for Index 462: rosario.
Closest hotel is Holiday Inn Rosario.
------------


In [10]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
43,napier,0.0,NZ,1.580018e+09,76.0,-39.48,176.92,77.00,4.00,The Nautilus Napier
80,arraial do cabo,0.0,BR,1.580018e+09,83.0,-22.97,-42.03,75.20,5.82,Pousada Porto Praia
163,ilhabela,0.0,BR,1.580018e+09,75.0,-23.78,-45.36,74.30,6.91,Ilha Flat Hotel
227,lalmohan,0.0,BD,1.580018e+09,28.0,22.34,90.74,73.74,7.18,Sundorbon courier service
267,bijni,0.0,IN,1.580018e+09,39.0,26.50,90.70,70.11,7.83,Bajwi Hotel Cum Dhaba
297,lambare,0.0,PY,1.580018e+09,88.0,-25.35,-57.61,71.60,3.36,Depto Tipo Loft Moderno Balcón
317,san rafael,0.0,AR,1.580018e+09,34.0,-34.62,-68.33,73.62,0.87,Hotel San Rafael
462,rosario,0.0,AR,1.580018e+09,78.0,-32.95,-60.64,79.00,3.85,Holiday Inn Rosario


In [13]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))